<a href="https://colab.research.google.com/github/BenaJovi/CSCI164/blob/main/Problem_Solving_w_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sliding Tile Puzzle: 3x3 and 4x4

This notebook expands the original 3x3 puzzle to support the 4x4 version and tests both with BFS and A* (with Misplaced Tile and Manhattan heuristics). For each case, we document:
- Start State
- Solution path
- Length of the solution
- Nodes expanded


In [ ]:

import heapq
import time
import random
from collections import deque
import gc


In [ ]:

class Puzzle:
    def __init__(self, state, size):
        self.state = state
        self.size = size
        self.goal = list(range(1, size * size)) + [0]

    def get_moves(self):
        idx = self.state.index(0)
        row, col = divmod(idx, self.size)
        moves = []
        if row > 0: moves.append(-self.size)
        if row < self.size - 1: moves.append(self.size)
        if col > 0: moves.append(-1)
        if col < self.size - 1: moves.append(1)
        return moves

    def move(self, move):
        idx = self.state.index(0)
        target = idx + move
        new_state = self.state[:]
        new_state[idx], new_state[target] = new_state[target], new_state[idx]
        return Puzzle(new_state, self.size)

    def is_goal(self):
        return self.state == self.goal

    def __hash__(self):
        return hash(tuple(self.state))

    def __eq__(self, other):
        return self.state == other.state

    def __str__(self):
        return "\n".join(
            [" ".join(f"{self.state[i * self.size + j]:2}" for j in range(self.size)) for i in range(self.size)]
        )


In [ ]:

def h_misplaced(puzzle):
    return sum(1 for i in range(len(puzzle.state)) if puzzle.state[i] != 0 and puzzle.state[i] != puzzle.goal[i])

def h_manhattan(puzzle):
    distance = 0
    for i, tile in enumerate(puzzle.state):
        if tile != 0:
            goal_i = puzzle.goal.index(tile)
            row1, col1 = divmod(i, puzzle.size)
            row2, col2 = divmod(goal_i, puzzle.size)
            distance += abs(row1 - row2) + abs(col1 - col2)
    return distance


In [ ]:
import itertools

def bfs(start):
    frontier = deque([start])
    visited = set()
    parent = {tuple(start.state): None}
    nodes_expanded = 0

    while frontier:
        current = frontier.popleft()
        nodes_expanded += 1
        if current.is_goal():
            return reconstruct_path(parent, current), nodes_expanded

        for move in current.get_moves():
            neighbor = current.move(move)
            if tuple(neighbor.state) not in parent:
                parent[tuple(neighbor.state)] = current
                frontier.append(neighbor)
    return None, nodes_expanded

def astar(start, heuristic):
    frontier = []
    counter = itertools.count()  # unique sequence count for tie-breaking
    heapq.heappush(frontier, (heuristic(start), 0, next(counter), start))
    parent = {tuple(start.state): None}
    g_cost = {tuple(start.state): 0}
    nodes_expanded = 0

    while frontier:
        _, cost, _, current = heapq.heappop(frontier)
        nodes_expanded += 1
        if current.is_goal():
            return reconstruct_path(parent, current), nodes_expanded

        for move in current.get_moves():
            neighbor = current.move(move)
            neighbor_state = tuple(neighbor.state)
            tentative_g = g_cost[tuple(current.state)] + 1

            if neighbor_state not in g_cost or tentative_g < g_cost[neighbor_state]:
                g_cost[neighbor_state] = tentative_g
                f = tentative_g + heuristic(neighbor)
                heapq.heappush(frontier, (f, tentative_g, next(counter), neighbor))
                parent[neighbor_state] = current

    return None, nodes_expanded

def reconstruct_path(parent, current):
    path = []
    while current is not None:
        path.append(current)
        current = parent[tuple(current.state)]
    path.reverse()
    return path


In [ ]:

def random_walk(size, steps):
    puzzle = Puzzle(list(range(1, size * size)) + [0], size)
    for _ in range(steps):
        puzzle = puzzle.move(random.choice(puzzle.get_moves()))
    return puzzle


In [ ]:

def run_and_display():
    steps_list = [5, 10, 20, 40, 80]
    sizes = [3, 4]
    algorithms = {
        'BFS': bfs,
        'A*_Misplaced': lambda p: astar(p, h_misplaced),
        'A*_Manhattan': lambda p: astar(p, h_manhattan)
    }

    for size in sizes:
        for steps in steps_list:
            for trial in range(3):
                print(f"\n=== Puzzle {size}x{size} | Walk: {steps} steps | Trial {trial + 1} ===")
                puzzle = random_walk(size, steps)
                print("Start State:")
                print(puzzle)

                for name, algo in algorithms.items():
                    if name == "BFS" and size == 4 and steps > 20:
                        print(f"Skipping {name} for 4x4 at {steps} steps (too large)")
                        continue

                    print(f"\n{name} Results:")
                    start_time = time.time()
                    path, expanded = algo(puzzle)
                    duration = round(time.time() - start_time, 4)

                    if path:
                        print(f"Solved in {len(path) - 1} moves")
                        print(f"Nodes Expanded: {expanded}")
                        print(f"Time: {duration}s")
                        print("Solution Path:")
                        for step in path:
                            print(step)
                            print()
                    else:
                        print("No solution found.")


In [ ]:


run_and_display()



=== Puzzle 3x3 | Walk: 5 steps | Trial 1 ===
Start State:
 1  0  2
 4  5  3
 7  8  6

BFS Results:
Solved in 3 moves
Nodes Expanded: 18
Time: 0.0002s
Solution Path:
 1  0  2
 4  5  3
 7  8  6

 1  2  0
 4  5  3
 7  8  6

 1  2  3
 4  5  0
 7  8  6

 1  2  3
 4  5  6
 7  8  0


A*_Misplaced Results:
Solved in 3 moves
Nodes Expanded: 4
Time: 0.0001s
Solution Path:
 1  0  2
 4  5  3
 7  8  6

 1  2  0
 4  5  3
 7  8  6

 1  2  3
 4  5  0
 7  8  6

 1  2  3
 4  5  6
 7  8  0


A*_Manhattan Results:
Solved in 3 moves
Nodes Expanded: 4
Time: 0.0004s
Solution Path:
 1  0  2
 4  5  3
 7  8  6

 1  2  0
 4  5  3
 7  8  6

 1  2  3
 4  5  0
 7  8  6

 1  2  3
 4  5  6
 7  8  0


=== Puzzle 3x3 | Walk: 5 steps | Trial 2 ===
Start State:
 1  2  3
 4  6  8
 7  0  5

BFS Results:
Solved in 5 moves
Nodes Expanded: 59
Time: 0.0005s
Solution Path:
 1  2  3
 4  6  8
 7  0  5

 1  2  3
 4  6  8
 7  5  0

 1  2  3
 4  6  0
 7  5  8

 1  2  3
 4  0  6
 7  5  8

 1  2  3
 4  5  6
 7  0  8

 1  2  3
 4  5  